# Workplace Things 1.1
### Defining your shifts and calculating work hours
###### Steps for excel file: Define Shifts, Shift Assignment and Overtime/Holiday Input, Contract Hours Input


In [1]:
import pandas as pd

In [3]:


# Load the Excel file
file_path = 'Dienstzeiten_JanC.xlsm'  # Update path accordingly
xl = pd.ExcelFile(file_path)

# Get sheet names to understand the structure
sheet_names = xl.sheet_names

# Load data from the first sheet to explore its structure
df_shifts = pd.read_excel(file_path, sheet_name='Schichten', engine='openpyxl')
df_overtime = pd.read_excel(file_path, sheet_name='Zusatzinformationen', engine='openpyxl')

df_shifts

,shift,duration,start,end,Sep,Oct,Nov,Dec,Jan,Feb,rotation,contract_hours,working_weeks,vacation_weeks
0,Urlaub,8.5000,07:30:00,16:30:00,NaN,7.0,7.0,NaN,NaN,NaN,winter,42.0,NaN,NaN
1,Früh,8.5000,07:30:00,16:30:00,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mittel,8.5000,11:30:00,20:30:00,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Spät,8.2500,15:00:00,23:45:00,5.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nacht,9.0000,22:30:00,08:00:00,3.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,WE-Tag,12.0000,07:30:00,20:15:00,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,WE-Mittel,12.0000,11:15:00,20:30:00,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,WE-Nacht,12.0000,19:30:00,08:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Beispielschicht 1,1.0166,10:10:00,11:11:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Beispielschicht 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df_overtime

,Month,Overtime Hours,Unnamed: 2,rotation
0,Sep,5.0,NaN,winter
1,Oct,NaN,NaN,NaN
2,Nov,NaN,NaN,NaN
3,Dec,NaN,NaN,NaN
4,Jan,NaN,NaN,NaN
5,Feb,NaN,NaN,NaN
6,Mar,NaN,NaN,NaN


In [5]:
#Formatting values
df_shifts['duration'] = pd.to_numeric(df_shifts['duration'], errors='coerce')
all_month_cols = ['Sep', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'Jun', 'Jul', 'Aug'] # Define all possible month columns
present_month_cols = [col for col in all_month_cols if col in df_shifts.columns] # Identify which month columns are actually present in the DataFrame
df_shifts[present_month_cols] = df_shifts[present_month_cols].apply(pd.to_numeric, errors='coerce')

summary_df = pd.DataFrame(columns=present_month_cols) # Initialize an empty DataFrame for summary

# Analysis 1: Monthly Work Hours: Calculating total work hours for each present month column
monthly_hours_all = df_shifts[present_month_cols].multiply(df_shifts['duration'], axis=0).sum()
summary_df.loc['Monatsstunden (Schichten + Urlaub)'] = monthly_hours_all

# Analysis 2: Monthly Work Hours (Excluding 'Urlaub'): Calculating total work hours (excluding 'Urlaub') for each present month column
monthly_hours_shifts_only = df_shifts.loc[df_shifts['shift'] != 'Urlaub', present_month_cols].multiply(df_shifts.loc[df_shifts['shift'] != 'Urlaub', 'duration'], axis=0).sum()
summary_df.loc['Monatsstunden (Nur Schichten)'] = monthly_hours_shifts_only

# Analysis 3: Monthly Work Hours (Only 'Urlaub'): Calculating total vacation hours for each present month column
monthly_hours_vacation = df_shifts.loc[df_shifts['shift'] == 'Urlaub', present_month_cols].multiply(df_shifts.loc[df_shifts['shift'] == 'Urlaub', 'duration'], axis=0).sum()
summary_df.loc['Monatsstunden (Urlaub)'] = monthly_hours_vacation

#Calculation for all weeks (using 4.35 weeks / month as approximation)
summary_df.loc['Wochenstunden (Schichten + Urlaub)'] = monthly_hours_all /4.35
summary_df.loc['Wochenstunden (Schichten + Urlaub)'] = monthly_hours_all /4.35

# Analysis 4: Actual Work Hours per Week (Considering Vacation)
# Calculating the percentage of vacation hours per month
percentage_vacation = monthly_hours_vacation / monthly_hours_all

# Calculating actual working weeks per month
actual_weeks_per_month = 4.35 - (percentage_vacation * 4.35)

# Calculating actual work hours per week (considering vacation)
actual_hours_per_week = monthly_hours_all / actual_weeks_per_month
summary_df.loc['Wochenstunden (Tatsächlich Gearbeitet)'] = actual_hours_per_week

# Displaying the summary DataFrame
print(summary_df)
summary_df


                                              Sep         Oct         Nov  \
Monatsstunden (Schichten + Urlaub)      199.75000  180.500000  194.500000   
Monatsstunden (Nur Schichten)           199.75000  121.000000  135.000000   
Monatsstunden (Urlaub)                    0.00000   59.500000   59.500000   
Wochenstunden (Schichten + Urlaub)       45.91954   41.494253   44.712644   
Wochenstunden (Tatsächlich Gearbeitet)   45.91954   61.898452   64.419327   

                                        Dec  Jan  Feb  
Monatsstunden (Schichten + Urlaub)      0.0  0.0  0.0  
Monatsstunden (Nur Schichten)           0.0  0.0  0.0  
Monatsstunden (Urlaub)                  0.0  0.0  0.0  
Wochenstunden (Schichten + Urlaub)      0.0  0.0  0.0  
Wochenstunden (Tatsächlich Gearbeitet)  NaN  NaN  NaN  


,Sep,Oct,Nov,Dec,Jan,Feb
Monatsstunden (Schichten + Urlaub),199.75000,180.500000,194.500000,0.0,0.0,0.0
Monatsstunden (Nur Schichten),199.75000,121.000000,135.000000,0.0,0.0,0.0
Monatsstunden (Urlaub),0.00000,59.500000,59.500000,0.0,0.0,0.0
Wochenstunden (Schichten + Urlaub),45.91954,41.494253,44.712644,0.0,0.0,0.0
Wochenstunden (Tatsächlich Gearbeitet),45.91954,61.898452,64.419327,NaN,NaN,NaN


In [ ]:
# Calculating total work hours for each month (Sep - Feb)
monthly_hours = df_shifts.iloc[:, 3:9].multiply(df_shifts['duration'], axis=0).sum()

# Displaying total work hours per month
print("Total Work Hours Per Month:")
print(monthly_hours)
